In [1]:
from tqdm import *

In [2]:
import os
import sys
import csv
import time
import argparse
import fnmatch
import librosa
import pandas as pd

from hparams import HParams as hp
from zipfile import ZipFile
from audio import preprocess
from utils import download_file
from datasets.tsp_speech import TSPSpeech
from datasets.mb_speech import MBSpeech
from datasets.lj_speech import LJSpeech

In [3]:
import sys
import time
import argparse


import numpy as np

import torch
import torch.nn.functional as F

# project imports
from models import Text2Mel
from hparams import HParams as hp
from logger import Logger
from utils import get_last_checkpoint_file_name, load_checkpoint, save_checkpoint
from datasets.data_loader import Text2MelDataLoader

In [5]:
dataset_path = "datasets/matsp_dataset/"
tsp_speech = TSPSpeech([],"matsp_dataset/", "matsp_metadata.csv")

In [6]:
tsp_speech.fnames

['191-0000',
 '191-0001',
 '191-0002',
 '191-0003',
 '191-0004',
 '191-0005',
 '191-0006',
 '191-0007',
 '191-0008',
 '191-0009',
 '191-0010',
 '191-0011',
 '191-0012',
 '191-0013',
 '191-0014',
 '191-0015',
 '191-0016',
 '191-0017',
 '191-0018',
 '191-0019',
 '191-0020',
 '191-0021',
 '191-0022',
 '191-0023',
 '191-0024',
 '191-0025',
 '191-0026',
 '191-0027',
 '191-0028',
 '191-0029',
 '191-0030',
 '191-0031',
 '191-0032',
 '191-0033',
 '191-0034',
 '191-0035',
 '191-0036',
 '191-0037',
 '191-0038',
 '191-0039',
 '191-0040',
 '191-0041',
 '191-0042',
 '191-0043',
 '191-0044',
 '191-0045',
 '191-0046',
 '191-0047',
 '191-0048',
 '191-0049',
 '191-0050',
 '191-0051',
 '191-0052',
 '191-0053',
 '191-0054',
 '191-0055',
 '191-0056',
 '191-0057',
 '191-0058',
 '191-0059',
 '191-0060',
 '191-0061',
 '191-0062',
 '191-0063',
 '191-0064',
 '191-0065',
 '191-0066',
 '191-0067',
 '191-0068',
 '191-0069',
 '191-0070',
 '191-0071',
 '191-0072',
 '191-0073',
 '233-0001',
 '233-0002',
 '233-0003',

In [7]:
# Do preprocessing
preprocess(dataset_path, tsp_speech)

  3%|▎         | 12/435 [00:02<01:42,  4.15it/s]


KeyboardInterrupt: 

In [ ]:
train_data_loader = Text2MelDataLoader(text2mel_dataset=TSPSpeech(['texts', 'mels', 'mel_gates']), batch_size=2,
                                       mode='train')

In [ ]:
next(iter(train_data_loader))

In [ ]:
TSPSpeech(['texts', 'mels', 'mel_gates'])

In [ ]:
testing = np.load("datasets/TSP_MD_Speaker/mels/MD19_01.npy")
import matplotlib.pyplot as plt
plt.imshow(testing.T, origin='lower')

In [ ]:
testing = np.load("datasets/LJSpeech-1.1/mels/LJ001-0001.npy")
import matplotlib.pyplot as plt
plt.imshow(testing.T, origin='lower')

In [ ]:
import os
import re
import codecs
import unicodedata
import numpy as np

vocab = "PE abcdefghijklmnopqrstuvwxyz'.?"  # P: Padding, E: EOS.
char2idx = {char: idx for idx, char in enumerate(vocab)}
idx2char = {idx: char for idx, char in enumerate(vocab)}


def text_normalize(text):
    text = ''.join(char for char in unicodedata.normalize('NFD', text)
                   if unicodedata.category(char) != 'Mn')  # Strip accents

    text = text.lower()
    text = re.sub("[^{}]".format(vocab), " ", text)
    text = re.sub("[ ]+", " ", text)
    return text


def read_metadata(metadata_file):
    fnames, text_lengths, texts = [], [], []
    print(fnames)
    transcript = os.path.join(metadata_file)
    lines = codecs.open(transcript, 'r', 'utf-8-sig').readlines()
    for line in lines:
        fname, _, text = line.strip().split("|")
        print(fname)
        fnames.append(fname)
        print(fnames)
        text = text_normalize(text) + "E"  # E: EOS
        text = [char2idx[char] for char in text]
        text_lengths.append(len(text))
        texts.append(np.array(text, np.long))
    print(fnames, text_lengths, texts, sep="\n")
    return fnames, text_lengths, texts


def get_test_data(sentences, max_n):
    normalized_sentences = [text_normalize(line).strip() + "E" for line in sentences]  # text normalization, E: EOS
    texts = np.zeros((len(normalized_sentences), max_n + 1), np.long)
    for i, sent in enumerate(normalized_sentences):
        texts[i, :len(sent)] = [char2idx[char] for char in sent]
    return texts


In [ ]:
os.getcwd()

In [ ]:
temp = read_metadata('datasets/TSP_M_Speakers/TSP-M_metadata.csv')